### Aşamalara chart print fonksiyonu koymaca.

In [1]:
import os
import pickle
import itertools
import pandas as pd

from matplotlib import colors
import matplotlib.pyplot as plt

from map_engine.map_generator import map

from quest_engine.quest_generator import *
from quest_engine.freytags_fitness import freytags

from encounter_engine.world_encounters import *

from character_engine.character import *

In [2]:
#nw = map()
#
#nw.populate_map(n_locations=1024, name="terrain", relaxed=True, k=100)
#nw.populate_map(n_locations=4096, name="civilisation", relaxed=True, k=250)
#nw.populate_map(n_locations=36, name="story_act", relaxed=True, k=250)
#nw.populate_map(n_locations=1024, name="threat", relaxed=True, k=100)
#
#nw.attribute_view(seed_1=24, seed_2=34,
#                  map_name="terrain", view_name="terrain")
#nw.attribute_view(seed_1=98, seed_2=50, map_name="civilisation",
#                  view_name="civilisation", double=False)
#nw.attribute_view(seed_1=34, seed_2=34, map_name="threat",
#                  view_name="threat", double=False)
#nw.attribute_view(seed_1=16, seed_2=16, map_name="story_act",
#                  view_name="story_act", double=False)
#
#nw.attribute_centroids("civilisation")
#nw.attribute_centroids("terrain")
##nw.land_mask()
#
#with open('logs/map_nw', 'wb') as f:
#    pickle.dump(nw, f)

with open("logs/map_nw", "rb") as f:
    nw = pickle.load(f)

nw.land_mask()

In [7]:
pop = quest_pop(nw,act=0, pop_size=10, steps=3)
ind = pop.evolve(gens=1,early_stop=False, mu_p=.1, xo="pmx", mutation="random_point", fancy_save=True,print_it=True)

Best ind in gen 1 is -8524.236149077875
Best ind in gen 2 is -8794.746531920713
Best ind in gen 3 is -8524.236149077875
Best ind in gen 4 is -8524.236149077875
Best ind in gen 5 is -8524.236149077875
Best ind in gen 6 is -8524.236149077875
Best ind in gen 7 is -5524.236149077875
Best ind in gen 8 is -5524.236149077875
Best ind in gen 9 is -3437.9757984181315
Best ind in gen 10 is -3437.9757984181315
Best ind in gen 11 is -3437.9757984181315
Best ind in gen 12 is -3437.9757984181315
Best ind in gen 13 is -3437.9757984181315
Best ind in gen 14 is -3424.853445319677
Best ind in gen 15 is -3424.853445319677
Best ind in gen 16 is -3424.853445319677
Best ind in gen 17 is -3424.853445319677
Best ind in gen 18 is -3424.853445319677
Best ind in gen 19 is -3424.853445319677
Best ind in gen 20 is -3424.853445319677
Best ind in gen 21 is -2437.9757984181315
Best ind in gen 22 is -2424.853445319677
Best ind in gen 23 is -2424.853445319677
Best ind in gen 24 is -2424.853445319677
Best ind in gen 25 

In [ ]:
cmap = colors.ListedColormap(
    ["blue", "white", "yellowgreen", "khaki", "lawngreen", "slategrey", "darkgreen"])

i = 1
for gen in pop.fancy:
    fig, ax = plt.subplots()
    fig.set_dpi(50)
    fig.set_size_inches(12, 12)
    ax.imshow(nw.views["terrain"].T, cmap=cmap, alpha=.66)
    for q in gen:
        color = (random.random(), random.random(), random.random())
        ax.scatter([i[0] for i in q], [i[1] for i in q], color="black",marker="*", s=15)
        ax.plot([i[0] for i in q], [i[1] for i in q], color=color,linestyle="dotted",linewidth=2)
        ax.set_title("Generation: {}".format(i))
    fig.savefig(f"fancy/{i}.png")
    i += 1





In [ ]:
for j in range(50):

    fig, ax = plt.subplots()
    fig.set_dpi(150)
    fig.set_size_inches(15, 15)
    ax.imshow(nw.views["terrain"].T, cmap=cmap, alpha=.50)

    q = ind.path
    
    ax.scatter([i[0] for i in q], [i[1] for i in q], color="black",marker="*", s=20)
    ax.plot([i[0] for i in q], [i[1] for i in q], color="purple",linewidth=3)
    ax.set_title("Generation: {}, Quest Found".format(i), size=20)
    fig.savefig(f"fancy/{924+j}.png")

In [ ]:
print(ind.path_threat)

ind.check_threat()

In [ ]:

muts = ["complete","single_point","random_point"]
xos = ["pmx","single_point","ar"]
mups =  [.5]

for param in [p for p in itertools.product(*[muts,xos,mups])]:

            params = {
                "map"           :   nw,
                "mutation"      :   param[0],
                "xo"            :   param[1],
                "mu_p"          :   param[2],
                "gens"          :   500,
                "shelf_size"    :   50,
                "pop_size"      :   100,
                "print_it"      :   False,
                "brute_force"   :   False,
                "log"           :   True,
                "early_stop"    :   True,
            }

            ql = quest_library(params)

In [ ]:
params = {
                "map"           :   nw,
                "mutation"      :   "random_point",
                "xo"            :   "pmx",
                "mu_p"          :   .5,
                "gens"          :   500,
                "shelf_size"    :   1,
                "pop_size"      :   100,
                "print_it"      :   True,
                "brute_force"   :   False,
                "log"           :   False,
                "early_stop"    :   False,
            }

a = quest_library(params)

In [ ]:
nw.printer(quest_line = ql_1)

In [ ]:

path = "logs/"
files = os.listdir(path)
files = [f for f in files if f.endswith(".csv")]

df = pd.concat([pd.read_csv(path+f) for f in files])


for f in files:
    os.remove(path+f)
    
df.to_csv("logs/merged.csv", index=False)




In [ ]:
i = 1
fig, ax = plt.subplots()
fig.set_dpi(150)
fig.set_size_inches(15, 15)

cmap = colors.ListedColormap(
    ["blue", "white", "yellowgreen", "khaki", "lawngreen", "slategrey", "darkgreen"])

ax.imshow(nw.views["terrain"].T, cmap=cmap, alpha=.50)




for gen in pop.fancy:
    for q in gen:
        ax.scatter([i[0] for i in q], [i[1] for i in q], color="black",marker="*", s=15)
        ax.plot([i[0] for i in q], [i[1] for i in q], color="red",linestyle="dotted",linewidth=1)
        ax.set_title("Generation: " + str(i))
        fig.savefig(f"fancy/{i}.png")


In [ ]:
files

In [ ]:
import imageio
images = []
#files = []
#for filename in os.listdir("/Users/cetiners/Desktop/Thesis/human_error/fancy"):
#    if ".png" in filename:
#        files.append(filename[:-4])
#
#files = files.sort(key=float)

files = [f"{i}.png" for i in range(1,1024)]

for filename in files:
        
    images.append(imageio.imread("fancy/"+filename))
imageio.mimsave('/Users/cetiners/Desktop/Thesis/human_error/fancy/ohhhhyeaaah.gif', images)


In [ ]:
dc = {"a" : 2, "b" : 2}

In [ ]:
nw.meta.keys()

In [ ]:

fig, ax = plt.subplots()
fig.set_dpi(150)
fig.set_size_inches(15, 15)
for a in ['unblurred', 'final']:
    ax.imshow(nw.meta[a].T, alpha=.66)
ax.set_title(a)
#ax.scatter(x, y, c="black", s=15)
#ax.scatter(x_1, y_1, c="red", s=15)